In [56]:
from scipy.io import wavfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import random 
import os

In [66]:
df = pd.read_csv('ESC-50-master/meta/esc50.csv')
# Define categories and their sounds
categories = {
    'Animals': ['dog', 'cat'],
    'Natural soundscapes & water sounds': ['rain', 'sea_waves'],
    'Human sounds': ['coughing', 'footsteps', ],
    'Interior/domestic sounds': ['keyboard_typing', 'vacuum_cleaner',],
    'Exterior/urban noises': ['airplane', 'car_horn']
}
# Initialize dictionary to store sampled filenames for each category
sampled_sounds = {} 
 
# For each category, sample one sound from each subcategory
for category, sounds in categories.items():
    # Get all rows matching the sounds in this category
    category_df = df[df['category'].isin(sounds)]
    # Sample one from each sound type 
    samples = pd.concat([
        category_df[category_df['category'] == sound].sample(n=1)
        for sound in sounds
    ])
    # Store filenames in dictionary
    sampled_sounds[category] = samples['filename'].tolist()
    
print(sampled_sounds)




{'Animals': ['2-118072-A-0.wav', '4-133047-A-5.wav'], 'Natural soundscapes & water sounds': ['1-54958-A-10.wav', '5-200461-A-11.wav'], 'Human sounds': ['3-142604-A-24.wav', '2-209472-A-25.wav'], 'Interior/domestic sounds': ['1-94231-A-32.wav', '1-79236-A-36.wav'], 'Exterior/urban noises': ['2-160888-A-47.wav', '5-180156-B-43.wav']}


In [70]:
wave_list_data = [] 
base_path = 'ESC-50-master/audio/'
for category in sampled_sounds:
    for filename in sampled_sounds[category]:
        complete_path = os.path.join(base_path, filename)
        sample_rate, sound_data = wavfile.read(complete_path)
        wave_list_data.append(sound_data)